## GPS location data from mobile phones to study impacts of Bogota cable car opening

### Some background: 
- Flat part of bogota and some hills in the south that are now connected to the city center
- Providers (private company) that collect mobile phone data at a large scale (buy data for all of Colombia from and focus on region within the city)
- Now have two months worth of data for testing (have data from month after the opening of the cable car 2019, also a month from 2022)
- Goal: Do analysis to see if the month of data is good enough to be worth buying 2018 data (more expensive)
- How to know if it is good enough (2 key factors): How many observations appear in Bogota and what is their granularity and usefulness for tracking individual movements (e.g. can we see locations/movements between different neighborhoods?)

### Tasks: 
1) CALCULATE TOTAL NUMBER OF OBSERVATIONS IN BOGOTA
    - each observation is a ping, timestamped, individual identifiers for each device
2) REGULARITY OF OBSERVATIONS PER DEVICE OVER TIME
    - How long do I observe this device over the month? How often do I see each person per day?


### Notes from 2023-03-08: 
- Drop users seen for less than 5 days and with less than 60 pings. Also for the home determination, maybe change the "bed time" to 10pm to 530am or 6am (try 11pm-5am and 10pm-6am).
- Want to be able to determine the home locations of the people. 

#### Next steps:
- Plots of user loss as the cutoffs for pings and days are varied.
- Number of pings during hours when home locations could be computed (per user). How many people? (see above for cutoffs)
- Map of home locations, especially see the regions in the south if we have some users from there. Generate a heatmap. Want to see some granularity.

In [ ]:
from datetime import datetime as dt
import pandas as pd
import unidecode
from matplotlib import pyplot as plt
import os
import glob
import numpy as np
import pyarrow.parquet as pq
import geopandas as gpd
import skmob
import contextily as cx
import mobilkit
import pytz
from matplotlib import cm
import statsmodels.api as sm


import seaborn as sns
from skmob.measures.individual import home_location
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import statsmodels.formula.api as smf
import time
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import Polygon

#set to working directory in an .env file
%cd "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/"

### Data

Elena gave me these potential column identifiers for the data: `['device_id', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy',
       'timestamp', 'ip_address', 'device_os', 'os_version', 'user_agent',
       'country', 'source_id', 'publisher_id', 'app_id', 'location_context]`
Looking at the data it looks like the columns are actually:
`['device_id', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy',
       'timestamp', '?', 'device_os', 'os_version', 'user_agent',
       'country', 'source_id', 'publisher_id', 'app_id', 'location_context]`

### Load metadata

In [ ]:
meta_folder = "./metadata/"
bogota_geo = meta_folder + "poligonos-localidades.shp"


# read in the shapefile with the geolocation data for Bogota
zats=gpd.read_file(bogota_geo)


# get the first boundary component as a Polygon object to map Bogota
boundary_bgta = Polygon(list(zats.geometry.unary_union.boundary.geoms)[0].coords) 
x, y = boundary_bgta.exterior.xy

plt.figure(figsize=(4, 4))
plt.plot(x, y, color='dodgerblue', linewidth=1)
plt.fill(x, y, color='aliceblue')
plt.axis('equal')
plt.show()

### Load data and filter for a box encompassing the larger area of Bogota

Want to do this as a less computationally intensive way to quickly select the data that we may need/that may be of interest before proceeding, to make the computations more efficient

In [ ]:
def load_data(filepaths, initial_cols, sel_cols, final_cols): 
    """Load in the mobile data and specify the columns"""
    ddf = dd.read_csv(filepaths, names=initial_cols)
    ddf = ddf[sel_cols]
    ddf.columns = final_cols
    return ddf 

def preprocess_mobile(df): #needs work
    """Process timestamp to datetime for dataframe with a "datatime" column with timestamp values."""
    df["datetime"] = dd.to_datetime(df["datetime"], unit='ms', errors='coerce')
    df["datetime"] = df["datetime"].dt.tz_localize('UTC').dt.tz_convert('America/Bogota')
    df = df.compute() #convert dask to pandas df after preprocessing
    return df

def get_days(data_folder):
    """Assuming a directory organized as a month's worth of days with files in each directory like "day=01", etc """
    day_dirs = glob.glob((data_folder + "*/"))
    return day_dirs

def get_files(data_folder, day_dir):
    """Assuming a dir corresponding to and named for a day day_dir, (e.g. "day=01") within the data_folder with that day's mobile data files."""
    day = day_dir.split(data_folder)[1]
    filepaths = glob.glob((day_dir + '*[!*.gz]')) # select all the non-zipped mobile data files
    print(day, filepaths)
    return filepaths, day

def find_within(df, boundaries): 
    gdf = gpd.GeoDataFrame(df.index, geometry=gpd.points_from_xy(np.array(df["lng"]),np.array(df["lat"])))
    gdf = gdf.set_crs(epsg=4326)
    ind = np.where(gdf.geometry.within(boundary_bgta))[0]
    within_bound_df = df.iloc[ind,:].reset_index()
    return within_bound_df

def find_within_box(ddf, minlon , maxlon, minlat, maxlat):
    """Quick way to filter out points not in a particular rectangular region."""
    box=[minlon,minlat,maxlon,maxlat]
    filtered_ddf = mobilkit.loader.crop_spatial(ddf, box).reset_index()
    return filtered_ddf

def write_within_bounds(ddf, output_folder, day):
    ddf.compute().to_csv((output_folder + '/' + day.split('/')[0] + '_within_bogota.csv'), sep=',')
    
data_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/year=2019/month=1/"
day_dirs = get_days(data_folder)

initial_cols=['device_id', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy', 'timestamp',  'ip_address', 'device_os', 'country', 'unknown_2', 'geohash']
sel_cols = ["device_id","latitude","longitude","timestamp","geohash","horizontal_accuracy"]
final_cols = ["user_id","lat","lng","datetime","geohash","horizontal_accuracy"]
output_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_box_2019"

# boundary box that roughly captures the larger county of Bogota
minlon = -74.453
maxlon = -73.992
minlat = 3.727
maxlat = 4.835

for i in range(0,len(day_dirs)): 
    print(i)
    day_dir = day_dirs[i]
    filepaths, day = get_files(data_folder, day_dir)
    ddf = load_data(filepaths, initial_cols, sel_cols, final_cols)
    ddf = find_within_box(ddf, minlon , maxlon, minlat, maxlat)
    write_within_bounds(ddf, output_folder, day)

### Load data in the box around Bogota and further filter to find the pings that were a) detected near the stations of interest and b) within the neighborhoods of interest 

We want to restrict the users to the areas with pings near the stations as dinlineated by the shapefile of Bogota that Elena gave me and label the pings with the station/neighborhood information as well. We also want to find the users that may live near the stations to see if their travel patterns change.

#### Data from within 1km of the stations of interest

In [ ]:
output_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_1km_stations_2019/"
shapefile_path = meta_folder + "Stations_Buffer_1000.shp" #"poligonos-localidades.shp"

def find_within(shapefile_path, df):

    # read in the shapefile containing the regions
    regions = gpd.read_file(shapefile_path)
    
    # create a geopandas GeoDataFrame from the Dash DataFrame
    geometry = gpd.points_from_xy(df.lng, df.lat)
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # perform a spatial join to keep only the rows within the regions
    filtered_gdf = gpd.sjoin(gdf, regions, op='within')

    # convert back to a pandas DataFrame for use in Dash
    filtered_df = filtered_gdf.drop('geometry', axis=1).reset_index(drop=True)
    return filtered_df


In [ ]:
output_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_1km_stations_2019/"

filepaths = glob.glob((data_folder + "*.csv"))

for i in range(1,len(filepaths)): #len(day_dirs)
    filepath = filepaths[i]
    day = filepath.split(data_folder)[1].split('_within_bogota.csv')[0]
    ddf = dd.read_csv(filepath)[final_cols]
    df = ddf.compute()
    filtered_df = find_within(shapefile_path, df)
    filtered_df.to_csv(f"{output_folder}/{day}_regions_labelled.csv", sep=',')

#### Data from within the neighborhoods from the file of Bogota that Elena shared

In [ ]:
output_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_localidades_2019/"
shapefile_path = meta_folder + "poligonos-localidades.shp"

filepaths = glob.glob((data_folder + "*.csv"))

for i in range(0,len(filepaths)): #len(day_dirs)
    filepath = filepaths[i]
    day = filepath.split(data_folder)[1].split('_within_bogota.csv')[0]
    ddf = dd.read_csv(filepath)[final_cols]
    df = ddf.compute()
    filtered_df = find_within(shapefile_path, df)
    filtered_df.to_csv(f"{output_folder}/{day}_regions_labelled.csv", sep=',')

### Load the data for the pings near the stations to detect users with home locations nearby

In [ ]:
data_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_1km_stations_2019/"
users_w_ping_near_station_filepath = '/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_users_w_ping_bogota_1km_stations.csv'

def get_users(data_folder, output_filepath, user_col="user_id"):
    filepaths = glob.glob((data_folder + "*.csv"))
    ddf = dd.read_csv(filepaths)
    print(f"There are {len(ddf)} pings from {len(ddf[user_col].value_counts().index)} unique users from this dataset.")
    
    # write out the users that have a ping near one of the stations and use this list to filter the data for all the pings within the Bogota region
    users = pd.DataFrame({user_col: list(ddf[user_col].value_counts().index)})
    users.to_csv(output_filepath, index=False)

get_users(data_folder, users_w_ping_near_station_filepath)

users_w_ping_near_station = pd.read_csv(users_w_ping_near_station_filepath)
users_w_ping_near_station.head()

In [ ]:
# read in the data from the bogota area and restrict it to those users with at least one ping in the area of one of the stations
def get_user_list(user_file, user_col="user_id"):
    """Takes in a file (user_file) with a single column of user ids and exports a list of those users"""
    users = pd.read_csv(user_file)
    return list(users[user_col])

def write_filter_data_by_users(data_folder: str, users: list, output_filepath: str, user_col="user_id"):
    """Takes a dataframe and filters it to only include data from the specified users."""
    filepaths = glob.glob((data_folder + "*.csv"))
    ddf = dd.read_csv(filepaths)
    ddf = ddf[cols_to_keep]
    users_num_og = len(ddf[user_col].value_counts().index)
    df_specified_users = ddf[ddf[user_col].isin(users)].compute().reset_index()
    users_num_post = len(df_specified_users[user_col].value_counts().index)
    df_specified_users.to_csv(output_filepath, index=False)
    print(f"Wrote data for the {users_num_post} of a total of {users_num_og} users for this dataset.")
    #return df_specified_users

cols_to_keep = ['user_id', 'lat', 'lng', 'datetime', 'geohash', 'horizontal_accuracy', 'Nombre_de_l', 'Identificad']
data_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_localidades_2019/"
user_filtered_data_filepath = '/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_data_for_users_w_ping_1km_stations.csv'

users = get_user_list(users_w_ping_near_station_filepath)
write_filter_data_by_users(data_folder, users, user_filtered_data_filepath)

In [ ]:
# load in the data for the bogota users for whom we detected at least one ping near a station of interest for further filtering 
def calculate_filter_user_stats(data_filepath: str, cols: list, renamed_cols: list, output_filepath: str, min_pings: int, min_days: int): 
    ddf = dd.read_csv(data_filepath)
    ddf = ddf[cols]
    ddf.columns = renamed_cols
    ddf['datetime']=dd.to_datetime(ddf['datetime'],unit='ms', errors='coerce')
    ddf["datetime"]=ddf["datetime"].dt.tz_localize('UTC').dt.tz_convert('America/Bogota')
    users_stats_df = mobilkit.stats.userStats(ddf).compute()
    users_stats_df_filtered = users_stats_df[(users_stats_df['pings'] >= min_pings) & (users_stats_df['daysActive'] >= min_days)]
    print(f"Based on {min_pings} ping and {min_days} day mininum cutoffs, kept {len(users_stats_df_filtered)} of a total of {len(users_stats_df)} users for this dataset.")
    users_stats_df_filtered.to_csv(output_filepath, index=False)
    mobilkit.stats.plotUsersHist(users_stats_df, min_pings=min_pings, min_days=min_days)

output_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/"
min_pings, min_days = 60, 10
renamed_cols = ['uid', 'lat', 'lng', 'datetime', 'geohash', 'horizontal_accuracy', 'Nombre_de_l', 'Identificad']
user_dataquantity_filtered_filepath = f'{output_folder}jan2019_users_w_ping_bogota_1km_stations_{min_pings}minpings_{min_days}mindays_in_bogota_region.csv'
calculate_filter_user_stats(user_filtered_data_filepath, cols_to_keep, renamed_cols, user_dataquantity_filtered_filepath, min_pings=min_pings, min_days=min_days) 

In [ ]:
# filter the data for this final set of users to compute which ones have home locations near the stations: 
users = get_user_list(user_dataquantity_filtered_filepath, user_col='uid')
data_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_localidades_2019/"
user_dataquantity_filtered_data_filepath = f'/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_data_for_users_w_ping_1km_stations_{min_pings}pings_{min_days}days.csv'
write_filter_data_by_users(data_folder, users, user_dataquantity_filtered_data_filepath)

In [ ]:

def read_preprocess_dash(data_filepath, cols, renamed_cols):
    ddf = dd.read_csv(user_dataquantity_filtered_data_filepath)
    ddf = ddf[cols]
    ddf.columns = renamed_cols
    ddf['datetime']=dd.to_datetime(ddf['datetime'],unit='ms', errors='coerce')
    ddf["datetime"]=ddf["datetime"].dt.tz_localize('UTC').dt.tz_convert('America/Bogota')
    return ddf 

ddf = read_preprocess_dash(user_dataquantity_filtered_data_filepath, cols_to_keep, renamed_cols)
ddf.head()

In [ ]:
shapefile = meta_folder + "Stations_Buffer_1000.shp"

def find_home_work_locs(ddf, shapefile, home_hrs=(22.0, 6.0), work_hrs=(9.5,16.5)):
    regions = gpd.read_file(shapefile)
    ddf_w_zones, tessellation_gdf = mobilkit.spatial.tessellate(ddf,tesselation_shp=shapefile,filterAreas=True)
    ddf_w_zones = mobilkit.stats.userHomeWork(ddf_w_zones,
                                         homeHours=home_hrs,
                                         workHours=work_hrs)
    #these next two actually detect home locations and may take some time
    ddf_w_zones_stat= mobilkit.stats.userHomeWorkLocation(ddf_w_zones)
    df_hw_locs = ddf_w_zones_stat.compute()
    return ddf_w_zones, tessellation_gdf, df_hw_locs

ddf_w_zones, tessellation_gdf, df_hw_locs = find_home_work_locs(ddf, shapefile, home_hrs=(22.0, 6.0), work_hrs=(9.5,16.5))

In [ ]:
tessellation_gdf_w_home =pd.merge(tessellation_gdf,df_hw_locs.groupby(by="home_tile_ID").count().reset_index()[["home_tile_ID","home_pings"]],left_on="tile_ID",right_on="home_tile_ID")

### Lets count and visualize the number of users that live near the stations

Make nicer plots later on

In [ ]:
print('The breakdown of users living by different stations that we can detect is as follows:\n', tessellation_gdf_w_home[['Station', 'home_pings']])
print(f"There are {tessellation_gdf_w_home['home_pings'].sum()} users that live near the stations that pass the filtering conditions for min pings and days detected.")
tessellation_gdf_w_home

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
ax.set_ylim([4.3,5])
ax.set_ylim([4.5,4.6])
ax.set_xlim([-74.2,-74.05])
tessellation_gdf_w_home.plot(column="home_pings",ax=ax,legend=True,vmax=600)
plt.savefig('./station_zones_map_home_10pm_6am_jan2019_accurate.png', dpi=200)
plt.show()

In [ ]:
def filter_for_users_living_in_specified_regions(df, output_file_path_no_ext: str): 
    users_w_home_tile = df[df.home_tile_ID.notnull()]
    users_w_home_tile['uid'] = users_w_home_tile.index
    users_w_home_tile.to_csv(f"{output_file_path_no_ext}_w_hometile.csv",index=False)
    users_w_home_ping = df[df.home_pings > 0]
    users_w_home_ping['uid'] = users_w_home_ping.index
    users_w_home_ping.to_csv(f"{output_file_path_no_ext}_w_homeping.csv",index=False)
    print(f'Out of {len(df)} users, {len(users_w_home_tile)} have a home tile ID assigned, and {len(users_w_home_ping)} have a home ping in the regions specified.')
    return

output_file_path_no_ext = f'/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_users_w_ping_1km_stations_{min_pings}pings_{min_days}days'
filter_for_users_living_in_specified_regions(df_hw_locs, output_file_path_no_ext)

### Getting the final data for these users to work with

In [ ]:
data_folder = "/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/bogota_localidades_2019/"
output_file_path_no_ext = f'/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_users_w_ping_1km_stations_{min_pings}pings_{min_days}days'
min_pings, min_days = 60, 10

# filter the data for this final set of users to compute which ones have home locations near the stations: 
final_user_hometile_filepath = output_file_path_no_ext + '_w_hometile.csv'
users = get_user_list(final_user_hometile_filepath, user_col='uid')
final_filtered_hometile_filepath = f'/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_users_w_ping_1km_stations_{min_pings}pings_{min_days}days_w_hometile.csv'
#write_filter_data_by_users(data_folder, users, final_filtered_hometile_filepath)


#final_user_homeping_filepath = output_file_path_no_ext + '_w_homeping.csv'
users = get_user_list(final_user_homeping_filepath, user_col='uid')
final_filtered_homeping_filepath = f'/Users/emilyrobitschek/git/ETH/SPUR/mobile_data_colombia/data/jan2019_bogota_users_w_ping_1km_stations_{min_pings}pings_{min_days}days_w_homeping.csv'
#write_filter_data_by_users(data_folder, users, final_filtered_homeping_filepath)